# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

My solution enhances my week 1 exercise solution to provide technical questions and answers to programming languages that are supported in my programming school. Which includes: Python, JavaScript, Rust, Java, Ruby, and C++. Uses tools to verify supported programming languages.

In [38]:
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import gradio as gr
import sqlite3

In [33]:
# constants

MODEL = 'gpt-4o-mini'
DB = "languages.db"
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS languages (language TEXT PRIMARY KEY, support TEXT)')
    conn.commit()

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

system_message = """
You are a helpful assistance that takes a technical question,  
and responds with an short explanation on proggraming languages that are supported according to the level of support. 
Give short, courteous answers, no more than 2 sentences.
Always be accurate. If you don't know the answer, say so.
"""

In [18]:
#insert supported PL into DB
def set_programming_language(lang, support):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO languages (language, support) VALUES (?, ?) ON CONFLICT(language) DO UPDATE SET support = ?', (lang.lower(), support, support))
        conn.commit()

In [19]:
#set supported PL into DB
langs = {"python": "beginner", "javascript": "intermediate", "rust": "advanced", "java": "beginner", "ruby": "intermediate", "c++": "advanced"}
for lang, support in langs.items():
    set_programming_language(lang, support)

In [29]:
# get_programming language function to check if user desired PL is supported

def get_programming_language_support(lang):
    print(f"DATABASE TOOL CALLED: checking support for {lang}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT support FROM languages WHERE language = ?', (lang.lower(),))
        result = cursor.fetchone()
        return f"Support for {lang} is {result[0]}" if result else f"{lang} is not supported"


In [ ]:
get_programming_language_support("python")

In [ ]:
pl_function = {
    "name": "get_programming_language_support",
    "description": "Get the level of support for a programming language in my programming school.",
    "parameters": {
        "type": "object",
        "properties": {
            "programming_language": {
                "type": "string",
                "description": "The programming language that the user wants to know about",
            },
        },
        "required": ["programming_language"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": pl_function}]
tools

In [35]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_programming_language_support":
            arguments = json.loads(tool_call.function.arguments)
            lang = arguments.get('programming_language')
            support_details = get_programming_language_support(lang)
            responses.append({
                "role": "tool",
                "content": support_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [36]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()